In [6]:
from urllib.request import urlretrieve, urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
imageLocation = bs.find('img', {'alt': 'python-logo'})['src']
urlretrieve (imageLocation, 'logo.jpg')

('logo.jpg', <http.client.HTTPMessage at 0x7ff258aa2460>)

In [31]:
import os
from urllib.request import urlretrieve, urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup

downloadDir = 'downloaded'
baseUrl = 'https://pythonscraping.com/'
baseNetloc = urlparse(baseUrl).netloc

def getAbsoluteURL(source):
    if urlparse(baseUrl).netloc == '':
        return baseUrl + source
    return source

def getDownloadPath(fileUrl):
    parsed = urlparse(fileUrl)
    netloc = parsed.netloc.strip('/')
    path = parsed.path.strip('/')
    localfile = f'{downloadDir}/{netloc}/{path}'
    
    # Remove the filename from the path in order to 
    # make the directory structure leading up to it
    localpath = '/'.join(localfile.split('/')[:-1])
    if not os.path.exists(localpath):
        os.makedirs(localpath)
    return localfile

html = urlopen(baseUrl)
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src=True)

for download in downloadList:
    fileUrl = getAbsoluteURL(download['src'])
    if fileUrl is not None:
        try:
            urlretrieve(fileUrl, getDownloadPath(fileUrl))
            print(fileUrl)
        except Exception as e:
            print(f'Could not retrieve {fileUrl} Error: {e}')

https://pythonscraping.com/wp-includes/js/jquery/jquery.min.js?ver=3.6.3
https://pythonscraping.com/wp-includes/js/jquery/jquery-migrate.min.js?ver=3.4.0
https://pythonscraping.com/wp-content/plugins/pagelayer/js/combined.js?ver=1.5.9
https://pythonscraping.com/wp-content/uploads/2023/04/python-logo-e1681354047443.png
https://pythonscraping.com/wp-content/uploads/2021/08/home1.jpg
https://pythonscraping.com/wp-content/uploads/2021/08/logo01-e1681353135199.png
https://pythonscraping.com/wp-content/themes/popularfx/js/navigation.js?ver=1.2.0


In [32]:
import csv

csvFile = open('test.csv', 'w+')
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number plus 2', 'number times 2'))
    for i in range(10):
        writer.writerow( (i, i+2, i*2))
finally:
    csvFile.close()

In [37]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
# The main comparison table is currently the first table on the page
table = bs.find('table',{'class':'wikitable'})
rows = table.findAll('tr')
csvFile = open('editors.csv', 'wt+')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text().strip())
        writer.writerow(csvRow)
finally:
    csvFile.close()

In [39]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import random
import pymysql
import re

conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
                       user='root', passwd='password', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute('USE scraping')

def store(title, content):
    cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content))
    cur.connection.commit()

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org'+articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
try:
    while len(links) > 0:
         newArticle = links[random.randint(0, len(links)-1)].attrs['href']
         print(newArticle)
         links = getLinks(newArticle)
finally:
    cur.close()
    conn.close()

/var/folders/y6/jnf4yrtx1pg3y9tqb8fmhnrr0000gp/T/ipykernel_62159/983605081.py:13: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


/wiki/Atheism
/wiki/Ethical_monotheism
/wiki/Zoroastrianism
/wiki/Longhouse_Religion
/wiki/Theatre
/wiki/Wayback_Machine
/wiki/RECAP_US_Federal_Court_Documents_(collection)
/wiki/Associated_Whistleblowing_Press#Filtrala
/wiki/Welcome_to_Video_case
/wiki/University_of_Birmingham
/wiki/Leonard_Huxley_(physicist)
/wiki/Knight_Commander_of_the_Order_of_the_British_Empire
/wiki/Robert_Fulton_(Royal_Marines_officer)
/wiki/Varyl_Begg
/wiki/First_Sea_Lord
/wiki/Hugh_Pigot_(Royal_Navy_officer,_born_1722)
/wiki/George_Brydges_Rodney,_1st_Baron_Rodney
/wiki/Albert_Walsh
/wiki/Nova_Scotia
/wiki/List_of_Prime_Ministers_of_Canada
/wiki/Liberal-Conservative_Party
/wiki/Eglinton_(electoral_district)
/wiki/Mitchell_Sharp
/wiki/Joseph-%C3%89douard_Cauchon
/wiki/Leo_Housakos
/wiki/Jean_Chr%C3%A9tien
/wiki/Minister_of_National_Revenue
/wiki/Supreme_Court_of_Canada
/wiki/Ottawa_River
/wiki/Chaudi%C3%A8re_Falls
/wiki/Greely,_Ontario
/wiki/Greely_Elementary_School
/wiki/Cathy_Curry
/wiki/Kanata,_Ontario
/wik

KeyboardInterrupt: 

In [55]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pymysql
from random import shuffle

conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
                       user='root', passwd='password', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute('USE wikipedia')

def insertPageIfNotExists(url):
    cur.execute('SELECT id FROM pages WHERE url = %s LIMIT 1', (url))
    page = cur.fetchone()
    if not page:
        cur.execute('INSERT INTO pages (url) VALUES (%s)', (url))
        conn.commit()
        return cur.lastrowid
    else:
        return page[0]

def loadPages():
    cur.execute('SELECT url FROM pages')
    return [row[0] for row in cur.fetchall()]

def insertLink(fromPageId, toPageId):
    cur.execute('SELECT EXISTS(SELECT 1 FROM links WHERE fromPageId = %s AND toPageId = %s)',(int(fromPageId), int(toPageId)))
    if not cur.fetchone()[0]:
        cur.execute('INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)', 
                    (int(fromPageId), int(toPageId)))
        conn.commit()

def pageHasLinks(pageId):
    cur.execute('SELECT EXISTS(SELECT 1 FROM links WHERE fromPageId = %s)', (int(pageId)))
    return cur.fetchone()[0]


def getLinks(pageUrl, recursionLevel, pages):
    if recursionLevel > 4:
        return

    pageId = insertPageIfNotExists(pageUrl)
    html = urlopen(f'http://en.wikipedia.org{pageUrl}')
    bs = BeautifulSoup(html, 'html.parser')
    links = bs.findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    links = [link.attrs['href'] for link in links]

    for link in links:
        linkId = insertPageIfNotExists(link)
        insertLink(pageId, linkId)
        if not pageHasLinks(linkId):
            print(f'Getting {link}')
            pages.append(link)
            getLinks(link, recursionLevel+1, pages)
        else:
            print(f'Already fetched {link}')
        
        
getLinks('/wiki/Kevin_Bacon', 0, loadPages()) 
cur.close()
conn.close()

Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/Kevin_Bacon
Already fetched /wiki/Kevin_Bacon
Already fetched /wiki/Kevin_Bacon
Already fetched /wiki/Kevin_Bacon_(disambiguation)
Already fetched /wiki/Philadelphia
Already fetched /wiki/Kevin_Bacon_filmography
Getting /wiki/Kyra_Sedgwick
Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/Kyra_Sedgwick
Already fetched /wiki/Kyra_Sedgwick
Already fetched /wiki/Kyra_Sedgwick
Getting /wiki/University_of_Southern_California
Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/University_of_Southern_California
Already fetched /wiki/University_of_Southern_California
Already fetched /wiki/University_of_Southern_California
Getting /wiki/USC_(disambiguation)
Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Getting /wiki/USC
Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/USC
Already fetched

Getting /wiki/Francis_Lieber
Getting /wiki/Thomas_Cooper_(American_politician,_born_1759)
Getting /wiki/Joseph_LeConte
Getting /wiki/Forced_labour
Getting /wiki/Confederate_States_of_America
Getting /wiki/W._J._Whipper
Getting /wiki/Beaufort,_South_Carolina
Getting /wiki/Francis_Lewis_Cardozo
Getting /wiki/University_of_South_Carolina_Board_of_Trustees
Getting /wiki/Normal_school
Getting /wiki/Henry_E._Hayne
Getting /wiki/Secretary_of_State_of_South_Carolina
Getting /wiki/Old_Campus_District,_University_of_South_Carolina
Getting /wiki/National_Register_of_Historic_Places
Getting /wiki/University_of_South_Carolina_Library
Getting /wiki/Robert_Mills_(architect)
Getting /wiki/Koger_Center_for_the_Arts
Getting /wiki/Carolina_Coliseum
Getting /wiki/Colonial_Life_Arena
Getting /wiki/Carolina_Stadium
Getting /wiki/Olympic_Games
Getting /wiki/Williams-Brice_Stadium
Getting /wiki/Congaree_River
Getting /wiki/Five_Points_(Columbia)
Getting /wiki/Hoshuko
Getting /wiki/ACT_(examination)
Getting /w

Getting /wiki/Richard_Riley
Getting /wiki/United_States_Secretary_of_Education
Getting /wiki/Darius_Rucker
Getting /wiki/Hootie_%26_the_Blowfish
Getting /wiki/William_Timmons_(politician)
Getting /wiki/United_States_House_of_Representatives
Getting /wiki/South_Carolina%27s_4th_congressional_district
Getting /wiki/Joe_Wilson_(U.S._politician)
Getting /wiki/United_States_House_of_Representatives
Getting /wiki/South_Carolina%27s_2nd_congressional_district
Getting /wiki/Presidents_of_the_University_of_South_Carolina
Getting /wiki/Robert_L._Caslen
Getting /wiki/Harris_Pastides
Getting /wiki/Michael_Amiridis
Getting /wiki/University_of_South_Carolina_System
Getting /wiki/The_First_Year_Experience_Program
Getting /wiki/McMaster_School
Getting /wiki/Multiracial_Americans
Getting /wiki/Pell_grant
Getting /wiki/American_middle_class
Getting /wiki/ISBN_(identifier)
Getting /wiki/Greenwood,_South_Carolina
Getting /wiki/The_State_(Columbia,_South_Carolina)
Getting /wiki/Columbia,_South_Carolina
Get

Getting /wiki/University_of_Southern_Mississippi
Getting /wiki/University_of_Tennessee
Getting /wiki/University_of_Texas_at_Austin
Getting /wiki/Texas_A%26M_University
Getting /wiki/Tulane_University
Getting /wiki/Vanderbilt_University
Getting /wiki/University_of_Virginia
Getting /wiki/Virginia_Commonwealth_University
Getting /wiki/Virginia_Tech
Getting /wiki/Virginia_State_University
Getting /wiki/West_Virginia_University
Getting /wiki/College_of_William_%26_Mary
Getting /wiki/Idaho_State_University
Getting /wiki/Ohio_University
Getting /wiki/Southeastern_Conference
Getting /wiki/Florida_Gators
Getting /wiki/Georgia_Bulldogs
Getting /wiki/Kentucky_Wildcats
Getting /wiki/Missouri_Tigers
Getting /wiki/South_Carolina_Gamecocks
Getting /wiki/Tennessee_Volunteers
Getting /wiki/Vanderbilt_Commodores
Getting /wiki/Alabama_Crimson_Tide
Getting /wiki/Arkansas_Razorbacks
Getting /wiki/Auburn_Tigers
Getting /wiki/LSU_Tigers_and_Lady_Tigers
Getting /wiki/Ole_Miss_Rebels
Getting /wiki/Mississippi_

Getting /wiki/Colorado_Mesa_University
Getting /wiki/Colorado_Mountain_College
Getting /wiki/Colorado_Northwestern_Community_College
Getting /wiki/Colorado_School_of_Mines
Getting /wiki/Colorado_State_University
Getting /wiki/Colorado_State_University%E2%80%93Pueblo
Getting /wiki/Colorado_Technical_University
Getting /wiki/Community_College_of_Aurora
Getting /wiki/Community_College_of_Denver
Getting /wiki/Emily_Griffith_Opportunity_School
Getting /wiki/Fort_Lewis_College
Getting /wiki/Front_Range_Community_College
Getting /wiki/Iliff_School_of_Theology
Getting /wiki/Lamar_Community_College
Getting /wiki/Metropolitan_State_University_of_Denver
Getting /wiki/Morgan_Community_College
Getting /wiki/Naropa_University
Getting /wiki/Nazarene_Bible_College
Getting /wiki/Northeastern_Junior_College
Getting /wiki/Otero_College
Getting /wiki/Pikes_Peak_State_College
Getting /wiki/Pueblo_Community_College
Getting /wiki/Regis_University
Getting /wiki/Red_Rocks_Community_College
Getting /wiki/Rocky_

...

Getting /wiki/South_Eastern_Freeway
Getting /wiki/Qatar_Airways
Getting /wiki/Adelaide_Airport
Getting /wiki/Adelaide_Airport
Getting /wiki/Parafield_Airport
Getting /wiki/RAAF_Base_Edinburgh
Getting /wiki/University_of_Adelaide
Getting /wiki/Royal_Adelaide_Hospital
Getting /wiki/Teaching_hospital
Getting /wiki/Flinders_Medical_Centre
Getting /wiki/Northfield,_South_Australia
Getting /wiki/Glenside_Hospital_(Adelaide)
Getting /wiki/Women%27s_and_Children%27s_Hospital,_Adelaide
Getting /wiki/Queen_Elizabeth_Hospital,_Adelaide
Getting /wiki/Modbury_Hospital
Getting /wiki/Lyell_McEwin_Hospital
Getting /wiki/Adelaide_Community_Healthcare_Alliance
Getting /wiki/Sisters_of_the_Little_Company_of_Mary
Getting /wiki/East_End,_Adelaide
Getting /wiki/South_Australian_Health_and_Medical_Research_Institute
Getting /wiki/Proton_therapy
Getting /wiki/Women%27s_and_Children%27s_Hospital
Getting /wiki/South_Australian_Health_and_Medical_Research_Institute
Getting /wiki/North_Terrace,_Adelaide
Getting /

Getting /wiki/North_Adelaide_Golf_Course
Getting /wiki/Royal_Adelaide_Golf_Club
Getting /wiki/Santos_Stadium
Getting /wiki/Sidewinders_Speedway
Getting /wiki/South_Australia_Aquatic_and_Leisure_Centre
Getting /wiki/State_Hockey_Centre_(South_Australia)
Getting /wiki/Titanium_Security_Arena
Getting /wiki/Adelaide_Casino
Getting /wiki/Adelaide_Central_Market
Getting /wiki/Adelaide_Entertainment_Centre
Getting /wiki/Adelaide_Festival_Centre
Getting /wiki/Adelaide_Showground
Getting /wiki/Adelaide_Zoo
Getting /wiki/The_Beachouse
Getting /wiki/Thebarton_Theatre
Getting /wiki/Aldinga_Beach,_South_Australia
Getting /wiki/Brighton,_South_Australia
Getting /wiki/Christies_Beach,_South_Australia
Getting /wiki/Glenelg,_South_Australia
Getting /wiki/Henley_Beach,_South_Australia
Getting /wiki/Largs_Bay,_South_Australia
Getting /wiki/Maslin_Beach,_South_Australia
Getting /wiki/Moana,_South_Australia
Getting /wiki/North_Haven,_South_Australia
Getting /wiki/Seacliff,_South_Australia
Getting /wiki/Sem

Getting /wiki/Australia_women%27s_national_under-20_soccer_team
Getting /wiki/Australia_women%27s_national_under-17_soccer_team
Getting /wiki/Australia_women%27s_national_futsal_team
Getting /wiki/Australian_soccer_league_system
Getting /wiki/A-League_Men
Getting /wiki/National_Second_Division
Getting /wiki/National_Premier_Leagues
Getting /wiki/National_Premier_Leagues_Capital_Football
Getting /wiki/National_Premier_Leagues_NSW
Getting /wiki/National_Premier_Leagues_Northern_NSW
Getting /wiki/National_Premier_Leagues_Queensland
Getting /wiki/National_Premier_Leagues_South_Australia
Getting /wiki/National_Premier_Leagues_Tasmania
Getting /wiki/National_Premier_Leagues_Victoria
Getting /wiki/National_Premier_Leagues_Western_Australia
Getting /wiki/A-League_Women
Getting /wiki/National_Premier_Leagues_Women%27s
Getting /wiki/National_Premier_Leagues_NSW_Women%27s
Getting /wiki/National_Premier_Leagues_Victoria_Women
Getting /wiki/National_Premier_Leagues_WA_Women
Getting /wiki/List_of_Au

Getting /wiki/United_States_Army_Special_Forces
Getting /wiki/Iraqi_insurgency_(2003%E2%80%932011)
Getting /wiki/U.S._military
Getting /wiki/Heckler_%26_Koch_MP5
Getting /wiki/Heckler_%26_Koch_MP5
Getting /wiki/Blowback_operation
Getting /wiki/Magazine_(firearms)
Getting /wiki/Closed_bolt
Getting /wiki/Cartridge_(firearms)
Getting /wiki/.45_ACP
Getting /wiki/.40_S%26W
Getting /wiki/Stopping_power
Getting /wiki/Automatic_firearm
Getting /wiki/Rate_of_fire
Getting /wiki/.45_ACP#Plus_P
Getting /wiki/Polymer
Getting /wiki/Semi-automatic_firearm
Getting /wiki/Safety_(firearms)
Getting /wiki/Stock_(firearm)
Getting /wiki/Iron_sights#Aperture_sights
Getting /wiki/Picatinny_rail
Getting /wiki/Receiver_(firearms)
Getting /wiki/Forearm_(firearm_component)
Getting /wiki/Tactical_light
Getting /wiki/Laser_applications#Laser_sight
Getting /wiki/Vertical_forward_grip
Getting /wiki/U.S._Customs_and_Border_Protection
Getting /wiki/.45_ACP
Getting /wiki/.40_S%26W
Getting /wiki/9%C3%9719mm_Parabellum
Ge

KeyboardInterrupt: 

In [ ]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('The body of the email is here')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'ryan@pythonscraping.com'
msg['To'] = 'webmaster@pythonscraping.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

In [1]:
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

def sendMail(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] ='christmas_alerts@pythonscraping.com'
    msg['To'] = 'ryan@pythonscraping.com'

    s = smtplib.SMTP('localhost')
    s.send_message(msg)
    s.quit()

bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
while(bs.find('a', {'id':'answer'}).attrs['title'] == 'NO'):
    print('It is not Christmas yet.')
    time.sleep(3600)
    bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
sendMail('It\'s Christmas!', 
         'According to https://isitchristmas.com/, it is Christmas!')

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
'